# Koppelen van borden- en wegenbestanden
Wegenbestand wordt omgezet naar een directed graph met de relevante restrictie-type verkeersborden (type-C) gekoppeld aan elk wegsegment. Deze graph is geschikt voor een kortste pad algoritme. Deze dataset is het basis wegenbestand die gebruikt wordt voor diverse casussen binnen dit project.

In [1]:
import psycopg2
try:
    # get secret values from keyvault
    postgres_pw = TokenLibrary.getSecret("redacted", "redacted", "redacted")
    # make connection
    conn = psycopg2.connect(dbname="rdt_dev",user="redacted",host="redacted",password=postgres_pw)
    conn.autocommit = True
    cur=conn.cursor()
    print("Database connected")
except:
    print("I am unable to connect to the database")

In [ ]:
# subselection_string="('Noord-Holland','Flevoland')"
subselection_string="('Noord-Holland','Flevoland','Utrecht')"

Selectie van gemeentes in het interessegebied. Sommige koppelingen schalen zeer nadelig, door opslitsing per gemeente kan de rekentijd acceptabel gehouden worden

In [ ]:
sql=f'''
SELECT  g.statcode
FROM    stg_ndw_wegen.cbs_gemeentegrenzen_2023 g,
        stg_ndw_wegen.cbs_provinciegrenzen_2023 p
WHERE   ST_Within(g.geom,p.geom)
AND     p.statnaam IN {subselection_string}'''
cur.execute(sql)
data=cur.fetchall()
data = [item for sublist in data for item in sublist]
# print(data)

## Make subselection

In [ ]:
sql=f'''drop table if exists stg_ndw_wegen.nwb_raw_subselection;
CREATE TABLE stg_ndw_wegen.nwb_raw_subselection
AS (
    select w.*
    FROM (
            SELECT  *, ST_GeomFromText(shape,28992) as geom
            FROM    stg_ndw_wegen.nwb_raw
            --WHERE   bst_code NOT IN ('PST','DST','BU','OVB','CADO','VV','FP','VP','RP') -- Zie https://downloads.rijkswaterstaatdata.nl/nwb-wegen/productinfo/beschrijvende_documentatie/Gebruikershandleiding_NWB_Wegen_v1.1.docx
            WHERE   bst_code NOT IN ('DST','CADO','VV','FP','VP','RP') -- Zie https://downloads.rijkswaterstaatdata.nl/nwb-wegen/productinfo/beschrijvende_documentatie/Gebruikershandleiding_NWB_Wegen_v1.1.docx
            OR      bst_code IS NULL
        ) as w
        ,(
            SELECT  ST_Union(ARRAY_AGG(geom)) as geom
            FROM    stg_ndw_wegen.cbs_provinciegrenzen_2023 c
            WHERE   statnaam IN {subselection_string}
            GROUP BY rubriek
        ) as p
    WHERE ST_Within(w.geom,p.geom)
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'stg_ndw_wegen'::VARCHAR,'nwb_raw_subselection'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'subset'::VARCHAR);'''
cur.execute(sql_meta)

In [ ]:
sql=f'''drop table if exists stg_ndw_borden.ndw_bebording_raw_geom_subselection;
CREATE TABLE stg_ndw_borden.ndw_bebording_raw_geom_subselection
AS (
    SELECT bc.*
    FROM (
            SELECT  *, ST_transform(ST_SetSRID(ST_MakePoint("location.wgs84.longitude"::float, "location.wgs84.latitude"::float), 4326),28992) as geom
            FROM    stg_ndw_borden.ndw_bebording_raw_catc
            WHERE   black_code ~ '^[-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?$' --regexp voor numerieke waarden
            OR      black_code IS NULL
        ) as bc
        ,(
            SELECT ST_Union(ARRAY_AGG(geom)) as geom
            FROM stg_ndw_wegen.cbs_provinciegrenzen_2023
            WHERE statnaam IN {subselection_string}
            GROUP BY rubriek
        ) as p
    WHERE ST_Within(bc.geom,p.geom)
    );'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'stg_ndw_borden'::VARCHAR,'ndw_bebording_raw_geom_subselection'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'subset'::VARCHAR);'''
cur.execute(sql_meta)

## Wegenbestand

In [ ]:
sql='''drop table if exists prd_ndw_wegen.nwb_wegvakken_directed;
CREATE TABLE prd_ndw_wegen.nwb_wegvakken_directed
AS (
    select w.wvk_id*-1 as id,
        w.jte_id_beg  as source,
        w.jte_id_end as target,
        frc as frc,
        1 as speed,  
        case 
            when ROUND(degrees(ST_Azimuth(ST_StartPoint(st_linemerge(w.geom)), ST_ENDPOINT(st_linemerge(w.geom))))/90)::int = 0 then 'N'
            when ROUND(degrees(ST_Azimuth(ST_StartPoint(st_linemerge(w.geom)), ST_ENDPOINT(st_linemerge(w.geom))))/90)::int = 1 then 'O'
            when ROUND(degrees(ST_Azimuth(ST_StartPoint(st_linemerge(w.geom)), ST_ENDPOINT(st_linemerge(w.geom))))/90)::int = 2 then 'Z'
            when ROUND(degrees(ST_Azimuth(ST_StartPoint(st_linemerge(w.geom)), ST_ENDPOINT(st_linemerge(w.geom))))/90)::int = 3 then 'W'
            when ROUND(degrees(ST_Azimuth(ST_StartPoint(st_linemerge(w.geom)), ST_ENDPOINT(st_linemerge(w.geom))))/90)::int = 4 then 'N'
            else 'X' 
        end as wegrichting,
        degrees(ST_Azimuth(ST_StartPoint(st_linemerge(w.geom)), ST_ENDPOINT(st_linemerge(w.geom))))::int as wegrichting_deg,
        w.wegbehsrt as RoadAuthorityType, w.wegbehnaam as RoadAuthorityName,
        geom as geom
    from stg_ndw_wegen.nwb_raw_subselection AS w
    where rijrichtng in ('B','H','O') 
    union all 
    select w.wvk_id as id,
        w.jte_id_end   as source, -- dit is bewust omgedraaid 
        w.jte_id_beg   as target, -- dit is bewust omgedraaid
        frc as frc, 
        1 as speed, 
        case 
            when ROUND(degrees(ST_Azimuth(ST_ENDPOINT(st_linemerge(w.geom)), ST_StartPoint(st_linemerge(w.geom))))/90)::int = 0 then 'N'
            when ROUND(degrees(ST_Azimuth(ST_ENDPOINT(st_linemerge(w.geom)), ST_StartPoint(st_linemerge(w.geom))))/90)::int = 1 then 'O'
            when ROUND(degrees(ST_Azimuth(ST_ENDPOINT(st_linemerge(w.geom)), ST_StartPoint(st_linemerge(w.geom))))/90)::int = 2 then 'Z'
            when ROUND(degrees(ST_Azimuth(ST_ENDPOINT(st_linemerge(w.geom)), ST_StartPoint(st_linemerge(w.geom))))/90)::int = 3 then 'W'
            when ROUND(degrees(ST_Azimuth(ST_ENDPOINT(st_linemerge(w.geom)), ST_StartPoint(st_linemerge(w.geom))))/90)::int = 4 then 'N'
            else 'X' 
        end as wegrichting,
        degrees(ST_Azimuth(ST_ENDPOINT(st_linemerge(w.geom)), ST_StartPoint(st_linemerge(w.geom))))::int as wegrichting_deg,
        w.wegbehsrt as RoadAuthorityType, w.wegbehnaam as RoadAuthorityName,
        st_reverse(geom) as geom -- dit is bewust omgedraaid
    from stg_ndw_wegen.nwb_raw_subselection AS w
    where rijrichtng in ('B','O')
    );'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_ndw_wegen'::VARCHAR,'nwb_wegvakken_directed'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'creating directed network'::VARCHAR);'''
cur.execute(sql_meta)
sql='''CREATE INDEX nwb_wegvakken_directed_geomidx
	ON prd_ndw_wegen.nwb_wegvakken_directed
  	USING GIST (geom);
    ANALYZE prd_ndw_wegen.nwb_wegvakken_directed;'''
cur.execute(sql)

In [ ]:
sql='''drop table if exists prd_ndw_wegen.nwb_wegvakken_nodes;
CREATE TABLE prd_ndw_wegen.nwb_wegvakken_nodes
AS (
    select DISTINCT node, geom
    FROM (
        select source as node,
            st_startpoint(st_linemerge(geom)) as geom
        from prd_ndw_wegen.nwb_wegvakken_directed
        union all 
        select  target as node,
            st_endpoint(st_linemerge(geom)) as geom
        from prd_ndw_wegen.nwb_wegvakken_directed
        ) as coll
    );'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_ndw_wegen'::VARCHAR,'nwb_wegvakken_nodes'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'creating directed network'::VARCHAR);'''
cur.execute(sql_meta)

In [ ]:
sql='''drop table if exists prd_ndw_wegen.nwb_wegvakken_oneway;
CREATE TABLE prd_ndw_wegen.nwb_wegvakken_oneway
AS (
    select w.wvk_id as id,
        w.jte_id_beg  as source,
        w.jte_id_end as target,
        frc as frc, 
        geom as geom
    from stg_ndw_wegen.nwb_raw_subselection AS w
    where rijrichtng ='H'
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_ndw_wegen'::VARCHAR,'nwb_wegvakken_oneway'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'creating directed network'::VARCHAR);'''
cur.execute(sql_meta)

## Bordenbestand

In [3]:
sql='''drop table if exists int_ndw_borden.ndw_bebording_verrijkt_1;
CREATE TABLE int_ndw_borden.ndw_bebording_verrijkt_1
AS (
	select b.id::INT as bord_id, b.ndw_id,
		-- case when b.textsigns is null then 'verbod' else 'onbekend'  end as geldigheid,
		case when text_signs = '{}' then 'verbod' when text_signs = '{""}' then 'verbod' else 'onbekend'  end as geldigheid,
		b.rvv_code,
		b.text_signs as bord_text,
		b.black_code as tekst_waarde,
		b."location.wgs84.latitude" as lat,
		b."location.wgs84.longitude" as lon, 
		b."location.side" as kijkrichting,
		CASE
			WHEN b."location.side" = 'N' then 0
			WHEN b."location.side" = 'O' then 90
			WHEN b."location.side" = 'Z' then 180
			WHEN b."location.side" = 'W' then 270
			WHEN b."location.side" = 'L' then -1
			WHEN b."location.side" = 'R' then -1
			ELSE -1
		END as kijkrichting_deg,
		b."location.road.name" as straatnaam,
		b."location.county.name" as gemeentenaam,
		b.organisation_id,
		b."details.image" as url_foto,
		concat ('https://www.google.com/maps/place/',b."location.wgs84.latitude",'+',b."location.wgs84.longitude") as google_url,
		b."location.road.wvk_id" as nwb_wegvak_id_volgens_ndw,
		cbs.postcode,
		b.geom as geom
	from stg_ndw_borden.ndw_bebording_raw_geom_subselection AS b
	LEFT JOIN   stg_ndw_wegen.cbs_pc6_2022 cbs
	ON          ST_Within(b.geom,cbs.geom)
	order by 	b.text_signs desc
	);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_ndw_borden'::VARCHAR,'ndw_bebording_verrijkt_1'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'data cleaning'::VARCHAR);'''
cur.execute(sql_meta)
sql='''CREATE INDEX ndw_bebording_verrijkt_1_geomidx
	ON int_ndw_borden.ndw_bebording_verrijkt_1
  	USING GIST (geom);
	ANALYZE int_ndw_borden.ndw_bebording_verrijkt_1;'''
cur.execute(sql)

In [4]:
sql='''drop table if exists int_ndw_wegen.nwb_wegvakken_directed_offset;
CREATE TABLE int_ndw_wegen.nwb_wegvakken_directed_offset
AS (
    SELECT w.*, ST_OffsetCurve(geom,-0.1) as geom_offset
    FROM prd_ndw_wegen.nwb_wegvakken_directed w
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_ndw_wegen'::VARCHAR,'nwb_wegvakken_directed_offset'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)

In [ ]:
for gm_code in data:
    # print(gm_code)
    sql_sub=f'''drop table if exists int_ndw_borden.ndw_bebording_verrijkt_1_sub;
    CREATE TABLE int_ndw_borden.ndw_bebording_verrijkt_1_sub
    AS (
        SELECT  b.*
        FROM    int_ndw_borden.ndw_bebording_verrijkt_1 b,
                stg_ndw_wegen.cbs_gemeentegrenzen_2023 g
        WHERE   ST_Within(b.geom,g.geom)
        AND     g.statcode = '{gm_code}'
    );'''
    cur.execute(sql_sub)
    sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_ndw_borden'::VARCHAR,'ndw_bebording_verrijkt_1_sub'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'subset'::VARCHAR);'''
    cur.execute(sql_meta)
    sql_sub2=f'''drop table if exists int_ndw_wegen.nwb_wegvakken_directed_offset_sub;
    CREATE TABLE int_ndw_wegen.nwb_wegvakken_directed_offset_sub
    AS (
        SELECT  w.*
        FROM    int_ndw_wegen.nwb_wegvakken_directed_offset w,
                stg_ndw_wegen.cbs_gemeentegrenzen_2023 g
        WHERE   ST_Within(w.geom,g.geom)
        AND     g.statcode = '{gm_code}'
    );'''
    cur.execute(sql_sub2)
    sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_ndw_wegen'::VARCHAR,'nwb_wegvakken_directed_offset_sub'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'subset'::VARCHAR);'''
    cur.execute(sql_meta)
    sql=f'''drop table if exists int_ndw_borden.ndw_bebording_verrijkt_2_{gm_code};
    CREATE TABLE int_ndw_borden.ndw_bebording_verrijkt_2_{gm_code}
    AS (
        select 
            bord_id, ndw_id, geldigheid, lat, lon, kijkrichting, kijkrichting_deg, straatnaam, gemeentenaam, postcode, organisation_id, url_foto, google_url, nwb_wegvak_id_volgens_ndw, geom,
            rvv_code as rvv_modelnummer, 
            bord_text as onderbord_tekst,
            tekst_waarde, 
            id as _wegvak_id_volgens_script,
            id as link_gevalideerd
        from (
            SELECT  b.*
                    ,CASE WHEN afstand<50 THEN roads_closest.ID::INT ELSE NULL END AS id
                    ,CASE WHEN afstand<50 THEN roads_closest.afstand::INT ELSE NULL END AS afstand
            FROM    int_ndw_borden.ndw_bebording_verrijkt_1_sub AS b
            CROSS JOIN LATERAL (
                SELECT  b.geom <-> n.geom_offset as afstand
                        ,n.ID
                FROM    int_ndw_wegen.nwb_wegvakken_directed_offset_sub as n
                ORDER BY afstand 
                LIMIT   1
            ) as roads_closest
        ) as sub
    );'''
    cur.execute(sql)
    sql_meta=f'''CALL update_metadata('rdt_dev'::VARCHAR,'int_ndw_borden'::VARCHAR,'ndw_bebording_verrijkt_2_{gm_code}'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'calculation'::VARCHAR);'''
    cur.execute(sql_meta)

In [ ]:
sql_union='''drop table if exists int_ndw_borden.ndw_bebording_verrijkt_2;
CREATE TABLE int_ndw_borden.ndw_bebording_verrijkt_2
AS ('''
gm_code_last = data[-1]
for gm_code in data:
    sql_union+=f'''select *
    from int_ndw_borden.ndw_bebording_verrijkt_2_{gm_code}
    '''
    if not gm_code == gm_code_last:
        sql_union+='''union all
        '''
sql_union+=''');'''
# print(sql_union)
cur.execute(sql_union)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_ndw_borden'::VARCHAR,'ndw_bebording_verrijkt_2'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'union'::VARCHAR);'''
cur.execute(sql_meta)

In [ ]:
sql='''drop table if exists int_ndw_borden.ndw_bebording_wegbeheerder;
CREATE TABLE int_ndw_borden.ndw_bebording_wegbeheerder
AS (
    WITH bv2 as (
        SELECT  *
                ,CASE
                    WHEN nwb_wegvak_id_volgens_ndw IS NOT NULL THEN abs(nwb_wegvak_id_volgens_ndw::INT) 
                    WHEN _wegvak_id_volgens_script IS NOT NULL THEN abs(_wegvak_id_volgens_script::INT)
                    ELSE NULL
                END as wvk
        From    int_ndw_borden.ndw_bebording_verrijkt_2
    )
    SELECT      bv2.*, w.wegbehsrt as organisation_type_based_on_road, w.wegbehcode as organisation_id_based_on_road, w.wegbehnaam as organisation_name_based_on_road
    From        bv2
    LEFT JOIN   stg_ndw_wegen.nwb_raw_subselection AS w
    ON          bv2.wvk = w.wvk_id
)'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_ndw_borden'::VARCHAR,'ndw_bebording_wegbeheerder'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)

In [2]:
sql='''drop table if exists prd_ndw_borden.ndw_bebording_verrijkt_3;
CREATE TABLE prd_ndw_borden.ndw_bebording_verrijkt_3
AS (
    SELECT  bord_id, ndw_id, geldigheid, lat, lon, kijkrichting, kijkrichting_deg, straatnaam, gemeentenaam, postcode, url_foto, google_url, b.geom
            ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            --rvv_code as rvv_modelnummer, bord_text as onderbord_tekst, tekst_waarde
            ,rvv_modelnummer, onderbord_tekst, tekst_waarde
            ,_wegvak_id_volgens_script
            ,nwb_wegvak_id_volgens_ndw
            ,CASE WHEN w.id IS NOT NULL THEN w.id ELSE NULL END as nwb_wegvak_id_volgens_ndw_directed
            ,w.id as link_gevalideerd --alleen NDW gegevens/koppeling gebruiken, ongeacht of het via het script te koppelen is
    FROM    int_ndw_borden.ndw_bebording_wegbeheerder as b
    -- Nieuwe methode, koppeling op basis van richting in graden
    LEFT JOIN LATERAL (
        SELECT  w_sub.id
        FROM    prd_ndw_wegen.nwb_wegvakken_directed as w_sub
        WHERE   (
            abs(w_sub.id)=b.nwb_wegvak_id_volgens_ndw::int 
            AND (
                abs(mod(b.kijkrichting_deg,360)-mod(w_sub.wegrichting_deg,360))<70 --koppelen als minder dan X graden verschil in richting
                OR
                abs(mod(b.kijkrichting_deg,360)-mod(w_sub.wegrichting_deg,360))>290 --koppelen als minder dan X graden verschil in richting
                )
            AND NOT (b.kijkrichting='R' OR b.kijkrichting='L'))
        OR		(
            abs(w_sub.id)=b.nwb_wegvak_id_volgens_ndw::int --kijkrichting R of L zijn altijd eenrichtingswegen
            AND (b.kijkrichting='R' OR b.kijkrichting='L'))
    ) as w ON true
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_ndw_borden'::VARCHAR,'ndw_bebording_verrijkt_3'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)

Bebording verrijkt 3 is opgeschoonde, gekoppelde, verrijkte data. Voor bepaalde toepassingen is (nagenoeg)

## Verkeersbesluitkoppeling
- Init table if not exists
- Fill/update based on raw data
- Use python-script to find missing values

In [ ]:
sql="""CREATE TABLE IF NOT EXISTS prd_ndw_borden.verkeersbesluiten_koppeltabel (
    bord_id         VARCHAR(50),
    ndw_id          VARCHAR(50),
    koop_id         VARCHAR(50),
    koop_titel      VARCHAR(255),
    koop_url        VARCHAR(255),
    datum_koppeling TIMESTAMPTZ,
    methode         VARCHAR(50),
    betrouwbaarheid FLOAT,
    deleted         BOOLEAN
);"""
# methode: 'ndw', 'user_input', 'automated_match'
# betrouwbaarheid: value 0-1. ndw & user_input geven hoge betrouwbaarheid, automated_match lager (afhankelijk van match type/methode)
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_ndw_borden'::VARCHAR,'verkeersbesluiten_koppeltabel'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'table init'::VARCHAR);'''
cur.execute(sql_meta)

In [ ]:
# check for verkeersbesluit URL in raw data that do not have a matching entry in the koppeltabel. Add them
# sql=""";"""
# cur.execute(sql)
# sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_ndw_borden'::VARCHAR,'verkeersbesluiten_koppeltabel'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'added new values from source'::VARCHAR);'''
# cur.execute(sql_meta)

In [ ]:
# any unmatched borden that have recently been updated: (prepare to) automatically match verkeersbesluit.
sql="""
DROP TABLE int_ndw_borden.verkeersbesluiten_inputjsons;
CREATE TABLE int_ndw_borden.verkeersbesluiten_inputjsons as (
    WITH    id_lookup as (
                SELECT DISTINCT bord_id::INT
                FROM            prd_ndw_borden.verkeersbesluiten_koppeltabel
            )
            ,contents as (
                SELECT      bv3.bord_id, rgs.ndw_id
                            ,rvv_modelnummer, organisation_name_based_on_road
                            ,CASE
                                WHEN organisation_type_based_on_road = 'G' THEN 'Gemeente'
                                WHEN organisation_type_based_on_road = 'P' THEN 'Provincie'
                                WHEN organisation_type_based_on_road = 'W' THEN 'Waterschap'
                                WHEN organisation_type_based_on_road = 'R' THEN 'Rijkswaterstaat'
                                ELSE 'Overig'
                            END as organisatietype
                            ,rgs."location.rd.x"::FLOAT as rd_x
                            ,rgs."location.rd.y"::FLOAT as rd_y
                            ,cbs.postcode
                            -- ,ARRAY[rgs."location.rd.x"::FLOAT, rgs."location.rd.y"::FLOAT] as locatie
                FROM        prd_ndw_borden.ndw_bebording_verrijkt_3 bv3
                LEFT JOIN   stg_ndw_borden.ndw_bebording_raw_geom_subselection rgs
                ON          bv3.bord_id = rgs.id
                LEFT JOIN   stg_ndw_wegen.cbs_pc6_2022 cbs
                ON          ST_Within(bv3.geom,cbs.geom)
                WHERE       bv3.bord_id NOT IN (SELECT * FROM id_lookup)
                -- LIMIT   100
            )
    SELECT  bord_id, ndw_id
            ,json_build_object(
                'beheerder',json_build_object(
                    'organisatietype',organisatietype,
                    'creator',organisation_name_based_on_road
                ),
                'verkeersbordcode',rvv_modelnummer,
                'organisatietype',organisatietype,
                'locatie',json_build_object(
                    'x',rd_x,
                    'y',rd_y,
                    'radius','30'
                )
            ) as input_json
            ,json_build_object(
                'beheerder',json_build_object(
                    'organisatietype',organisatietype,
                    'creator',organisation_name_based_on_road
                ),
                'verkeersbordcode',rvv_modelnummer,
                'organisatietype',organisatietype,
                'postcode',postcode
            ) as input_json_alt_postcode
    FROM    contents c
);"""
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_ndw_borden'::VARCHAR,'verkeersbesluiten_inputjsons'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'constructing searchqueries'::VARCHAR);'''
cur.execute(sql_meta)

# run pipeline PL_verkeersbesluiten_inputjasons further down the stack

## Gekoppeld wegen-/bordenbestand

In [3]:
sql='''-- verrijkte borden koppelen aan netwerk 
drop table if exists prd_ndw_borden.nwb_wegvakken_bebording;
CREATE TABLE prd_ndw_borden.nwb_wegvakken_bebording
AS (
	select
		sub.id::int,
		sub.source::int,
		sub.target::int,
		sub.speed as spd, 
		case
			when speed is null then -1
			when speed = 0 then -1
			else st_length(sub.geom)/(speed/3.6) 
		end as cost,
		c01,c06,c07,c07a,c07b,c08,c09,c10,c11,c12,c17, c18, c19, c20, c21,
		sub.wegrichting,
		sub.RoadAuthorityType, sub.RoadAuthorityName,
		sub.geom
	from (
		select  id , source, target,  speed, wegrichting, RoadAuthorityType, RoadAuthorityName, st_transform(geom,4326) as geom
		from prd_ndw_wegen.nwb_wegvakken_directed
	) as sub
	left join (
		select  vb.link_gevalideerd as linknr_vma,
			min(case when vb.rvv_modelnummer = 'C1' then 1 else NULL end) as c01,
			min(case when vb.rvv_modelnummer = 'C6' then 1 else NULL end) as c06,
			min(case when vb.rvv_modelnummer LIKE 'C7%' AND vb.rvv_modelnummer NOT LIKE 'C7A%' AND vb.rvv_modelnummer NOT LIKE 'C7B%' then 1 else NULL end) as c07,
			min(case when vb.rvv_modelnummer = 'C7A' then 1 else NULL end) as c07a,
			min(case when vb.rvv_modelnummer = 'C7B' then 1 else NULL end) as c07b,
			min(case when vb.rvv_modelnummer = 'C8' then 1 else NULL end) as c08,
			min(case when vb.rvv_modelnummer = 'C9' then 1 else NULL end) as c09,
			min(case when vb.rvv_modelnummer = 'C10' then 1 else NULL end) as c10,
			min(case when vb.rvv_modelnummer = 'C11' then 1 else NULL end) as c11,
			min(case when vb.rvv_modelnummer = 'C12' then 1 else NULL end) as c12,
			min(case when vb.rvv_modelnummer = 'C17' then REPLACE(vb.tekst_waarde,',','.')::float else NULL end) as c17,
			min(case when vb.rvv_modelnummer = 'C18' then REPLACE(vb.tekst_waarde,',','.')::float else NULL end) as c18,
			min(case when vb.rvv_modelnummer = 'C19' then REPLACE(vb.tekst_waarde,',','.')::float else NULL end) as c19,
			min(case when vb.rvv_modelnummer = 'C20' then REPLACE(vb.tekst_waarde,',','.')::float else NULL end) as c20,
			min(case when vb.rvv_modelnummer = 'C21' or vb.rvv_modelnummer = 'C21_ZB' then REPLACE(vb.tekst_waarde,',','.')::float else NULL end) as c21
		from  prd_ndw_borden.ndw_bebording_verrijkt_3 AS vb
		where  vb.link_gevalideerd  <> 0
		and geldigheid = 'verbod'
		group by  vb.link_gevalideerd
	) as borden
	on sub.id=borden.linknr_vma
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_ndw_borden'::VARCHAR,'nwb_wegvakken_bebording'::VARCHAR,NOW(),'RDT_NDW_wegenborden_verwerking_v2'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)
sql='''CREATE INDEX nwb_wegvakken_bebording_geomidx
	ON prd_ndw_borden.nwb_wegvakken_bebording
  	USING GIST (geom);
	ANALYZE prd_ndw_borden.nwb_wegvakken_bebording;'''
cur.execute(sql)